## The original datasets I have used are the two train_multi and test_multi csvs provided by Devon 

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import GridSearchCV

df1=pd.read_csv(r"C:\nydsa bootcamp slides\Projects\4\train_multi.csv")
df2=pd.read_csv(r"C:\nydsa bootcamp slides\Projects\4\test_multi.csv")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [138]:
train_df=df1.copy()
test_df=df2.copy()

for col in test_df:
    series=test_df[col]
    series[series.isna()]="unknown"
for col in train_df:
    series=train_df[col]
    series[series.isna()]="unknown"

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### 1 . Remove weight column

In [139]:
train_df.drop('weight',axis=1, inplace=True)
test_df.drop('weight',axis=1,inplace=True)

In [140]:
print (train_df.shape,test_df.shape)

(81414, 49) (20352, 49)


### 2. Remove 3 rows that "gender"="Unknow/Invalid value"

In [141]:
train_df.drop(train_df[train_df['gender']=="Unknown/Invalid"].index,axis=0,inplace=True)
test_df.drop(test_df[test_df['gender']=="Unknown/Invalid"].index,axis=0,inplace=True)

In [142]:
print (train_df.shape,test_df.shape)

(81411, 49) (20352, 49)


### 3. Drop rows that "race"="unknown"

In [143]:
train_df.drop(train_df[train_df['race']=="unknown"].index,axis=0,inplace=True)
test_df.drop(test_df[test_df['race']=="unknown"].index,axis=0,inplace=True)

In [144]:
print (train_df.shape,test_df.shape)

(79604, 49) (19888, 49)


### 4. Drop column "payer_code"

In [145]:
train_df.drop('payer_code',axis=1, inplace=True)
test_df.drop('payer_code',axis=1, inplace=True)

In [146]:
print (train_df.shape,test_df.shape)

(79604, 48) (19888, 48)


### 5. remove rows that are unknown in diag_1, diag_2 and diag_3
* originally diag_1 has 21 unknwon 
* diag_2 has 358 unknown
* diag_3 has 1423 unknown

Some rows may have both diag_1 and diag_2 as unknown , also some rows with unknown values got removed in the first 4 steps. 
Therefore, in total in this step we have 1440 rows removed

In [147]:
train_df.drop(train_df[train_df['diag_1']=="unknown"].index,axis=0,inplace=True)
test_df.drop(test_df[test_df['diag_1']=="unknown"].index,axis=0,inplace=True)

train_df.drop(train_df[train_df['diag_2']=="unknown"].index,axis=0,inplace=True)
test_df.drop(test_df[test_df['diag_2']=="unknown"].index,axis=0,inplace=True)

train_df.drop(train_df[train_df['diag_3']=="unknown"].index,axis=0,inplace=True)
test_df.drop(test_df[test_df['diag_3']=="unknown"].index,axis=0,inplace=True)

In [148]:
print (train_df.shape,test_df.shape)

(78447, 48) (19605, 48)


### 6. remove the following columns because of too few real data
* 'tolbutamide', 23 records
* 'glipizide-metformin', 13 records
* 'acetohexamide', 1 record
* 'troglitazone', 3 record
* 'examide', 0 record
* 'citoglipton', 0 record
* 'glimepiride-pioglitazone', 1 record
* 'metformin-rosiglitazone', 2 records
* 'metformin-pioglitazone', 1 record

In [149]:
train_df.drop(['tolbutamide','glipizide.metformin','acetohexamide','troglitazone','examide',
                'citoglipton','glimepiride.pioglitazone','metformin.rosiglitazone','metformin.pioglitazone'],axis=1, inplace=True)

test_df.drop(['tolbutamide','glipizide.metformin','acetohexamide','troglitazone','examide',
                'citoglipton','glimepiride.pioglitazone','metformin.rosiglitazone','metformin.pioglitazone'],axis=1, inplace=True)

In [150]:
print (train_df.shape,test_df.shape)

(78447, 39) (19605, 39)


### 7. Based on our discussion , remove "discharge_disposition_id"= "expired" rows (id = 11,19,20,21)
* 11 - 1606 rows
* 19 - 8 rows
* 20 - 2 rows
* 21 - 0 rows

In [151]:
train_df.drop(train_df[train_df['discharge_disposition_id']==11].index,axis=0,inplace=True)
test_df.drop(test_df[test_df['discharge_disposition_id']==11].index,axis=0,inplace=True)

train_df.drop(train_df[train_df['discharge_disposition_id']==19].index,axis=0,inplace=True)
test_df.drop(test_df[test_df['discharge_disposition_id']==19].index,axis=0,inplace=True)

train_df.drop(train_df[train_df['discharge_disposition_id']==20].index,axis=0,inplace=True)
test_df.drop(test_df[test_df['discharge_disposition_id']==20].index,axis=0,inplace=True)

In [152]:
print (train_df.shape,test_df.shape)

(77147, 39) (19289, 39)


### 8. Based on the paper, remove " discharge_disposition_id"="discharge to hospice" (id=13,14)
* 13-396 rows
* 14-368 rows

In [153]:
train_df.drop(train_df[train_df['discharge_disposition_id']==13].index,axis=0,inplace=True)
test_df.drop(test_df[test_df['discharge_disposition_id']==13].index,axis=0,inplace=True)

train_df.drop(train_df[train_df['discharge_disposition_id']==14].index,axis=0,inplace=True)
test_df.drop(test_df[test_df['discharge_disposition_id']==14].index,axis=0,inplace=True)

In [154]:
print (train_df.shape,test_df.shape)

(76519, 39) (19153, 39)


### 9. drop "encounter_id" and "patient_nbr" column
* in my understanding these are just  unique IDs for the patients

In [155]:
train_df.drop(['encounter_id','patient_nbr'],axis=1, inplace=True)
test_df.drop(['encounter_id','patient_nbr'],axis=1, inplace=True)

In [156]:
print (train_df.shape,test_df.shape)

(76519, 37) (19153, 37)


### 10. update "readmitted" column to "yes" or "no"
* "<30" replace with "yes"
* all others replace with "no"

In [157]:
train_df['readmitted']=train_df['readmitted'].replace(["<30",">30"],["Yes","NO"])
test_df['readmitted']=test_df['readmitted'].replace(["<30",">30"],["Yes","NO"])

In [158]:
set(test_df['readmitted'])

{'NO', 'Yes'}

### 11. We have around 800 different categories in each diag_ column, combine them to the 9 main categories based on the paper

In [159]:
len(set(train_df['diag_1']))

694

In [160]:
# The replace_diag function can be used to replace the ids in the diag_1,diag_2 and diag_3 to the 9 main categories
def replace_diag(df):
    Circulatory_list = [str(i) for i in range(350,460)]
    Respiratory_list = [str(i) for i in range(460,520)]
    Digestive_list = [str(i) for i in range(520,589)]    
    Injury_list = [str(i) for i in range(800,1000)]
    Musculoskeletal_list=[str(i) for i in range(710,740)]
    Genitourinary_list=[str(i) for i in range(589,630)]
    Neoplasms_list=[str(i) for i in range(140,240)]
    
    for i in df:
        if  (i in Circulatory_list) or i=='785':
            df.replace(i,'Circulatory',inplace=True)
        elif (i in Respiratory_list) or i =='786':
            df.replace(i,'Respiratory', inplace=True)
        elif (i in Digestive_list) or i =='787':
            df.replace(i,'Digestive', inplace=True)
        elif (i in Injury_list):
            df.replace(i,'Injury', inplace=True)
        elif (i in Musculoskeletal_list):
            df.replace(i,'Musculoskeletal', inplace=True)
        elif (i in Genitourinary_list) or i =='788':
            df.replace(i,'Genitourinary', inplace=True)
        elif (i in Neoplasms_list):
            df.replace(i,'Neoplasms', inplace=True)
        elif len(i)>3:
            df.replace(i,'Diabetes', inplace=True)
        else:
            df.replace(i,'Other',inplace=True)
            

In [ ]:
replace_diag(train_df['diag_1'])
replace_diag(test_df['diag_1'])
print (len(set(train_df['diag_1'])),len(set(test_df['diag_1'])))

In [ ]:
replace_diag(train_df['diag_2'])
replace_diag(test_df['diag_2'])
print (len(set(train_df['diag_2'])),len(set(test_df['diag_2'])))

In [ ]:
replace_diag(train_df['diag_3'])
replace_diag(test_df['diag_3'])
print (len(set(train_df['diag_3'])),len(set(test_df['diag_3'])))

### Okay, we get our origin dataset, save it as "cleantrain.csv" and "cleantest.csv" first. Then we move to encode the nominal columns 

In [ ]:
print (train_df.shape,test_df.shape)

In [ ]:
train_df.to_csv("cleantrain.csv")
test_df.to_csv("cleantest.csv")

* **


### 12. Encode the following columns for xgboost 
race,gender,age,medical_specialty,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,
chlorpropamide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,acarbose,miglitol,tolazamide,insulin,
glyburide-metformin,change,diabetesMed

In [8]:
#copy origin_df to encoder_df, so we don't make changes on origin_df
train=pd.read_csv('cleantrain.csv',index_col=0)
test=pd.read_csv('cleantest.csv',index_col=0)
lencoder=LabelEncoder()
ind="_labels"
encoder_list=['race','gender','age','medical_specialty','max_glu_serum','A1Cresult','metformin','repaglinide','nateglinide',
        'chlorpropamide','glimepiride','glipizide','glyburide','pioglitazone','rosiglitazone','acarbose','miglitol',
        'tolazamide','insulin','glyburide.metformin','change','diabetesMed','diag_1', 'diag_2', 'diag_3']

numeric_col=['admission_type_id','discharge_disposition_id','admission_source_id','time_in_hospital','num_lab_procedures',
            'num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses',
            'readmitted']

In [9]:
train.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'tolazamide',
       'insulin', 'glyburide.metformin', 'change', 'diabetesMed',
       'readmitted'],
      dtype='object')

In [10]:
for i in encoder_list:
    lencoder.fit(train[i])
    test[i] = test[i].map(lambda s: 'unknown' if s not in lencoder.classes_ else s)
    lencoder.classes_ = np.append(lencoder.classes_, 'unknown')
    
    train[i]=lencoder.transform(train[i]).reshape(-1,1)
    test[i]=lencoder.transform(test[i]).reshape(-1,1)

In [13]:
train['readmitted']=train['readmitted'].replace(["Yes","NO"],[1,0])
train.head(50)

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,...,pioglitazone,rosiglitazone,acarbose,miglitol,tolazamide,insulin,glyburide.metformin,change,diabetesMed,readmitted
1,2,0,1,1,1,7,3,71,59,0,...,1,1,1,1,0,3,1,0,1,0
2,2,1,3,1,1,7,2,71,44,1,...,1,1,1,1,0,3,1,0,1,0
3,2,1,4,1,1,7,1,71,51,0,...,1,1,1,1,0,2,1,0,1,0
4,2,1,5,2,1,2,3,71,31,6,...,1,1,1,1,0,2,1,1,1,0
5,2,1,6,3,1,2,4,71,70,1,...,1,1,1,1,0,2,1,0,1,0
6,2,1,7,1,1,7,5,71,73,0,...,1,1,1,1,0,1,1,1,1,0
7,2,0,9,3,3,4,12,18,33,3,...,1,2,1,1,0,2,1,0,1,0
8,0,1,6,2,1,4,7,71,62,0,...,1,1,1,1,0,2,1,0,1,1
9,2,1,8,1,6,7,10,11,55,1,...,1,1,1,1,0,2,1,1,1,0
10,0,0,6,3,1,2,1,71,49,5,...,1,1,1,1,0,2,1,1,1,0


In [15]:
test['readmitted']=test['readmitted'].replace(["Yes","NO"],[1,0])

In [16]:
# Save the label encoded dfs to csv files
train.to_csv('le_train.csv')
test.to_csv('le_test.csv')